# Job Market Analysis: Extracting and Analyzing Skills from Listings

In [1]:
from data_preparation import prepare_data

In [2]:
# Check information about the DataFrame
df = prepare_data()
df.head(5)

,index,job_description,job_title,skills,num_phrases,skills_text,num_words,unique_words,num_uniques
0,0,About the job\n \n \nAbout RevolutP...,Head of Product (Crypto Exchange),"[Crypto Exchange Product Management, Team Lead...",6,Crypto Exchange Product Management Team Leader...,15,"[Crypto, Stakeholder, Exchange, Leadership, Co...",12
1,1,About the job\n \n \nASI Reisen ist...,Head of Product (w/m/d),"[Team management, Team development, Process im...",13,Team management Team development Process impro...,22,"[methodologies, management, Monitoring, making...",18
2,2,About the job\n \n \nDepartment: Pr...,Head of Product - Voluum,"[Product Strategy, Team Leadership, Execution,...",13,Product Strategy Team Leadership Execution Mar...,28,"[Experience, Analytics, Research, Strategy, ta...",25
3,3,About the job\n \n \nDescriptionAbo...,Senior Product Owner,"[Live Casino product development, Game design ...",6,Live Casino product development Game design an...,16,"[testing, management, Live, design, Problem, P...",14
4,4,About the job\n \n \nProyecto ambic...,CPO-B2C- Remoto 100%,"[Product analysis and improvement, Sales strat...",5,Product analysis and improvement Sales strateg...,16,"[business, improvement, Translating, technical...",15


In [3]:
# Display first 5 rows
# pd.set_option("display.max_colwidth", 110)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   index            25 non-null     int64 
 1   job_description  25 non-null     object
 2   job_title        25 non-null     object
 3   skills           25 non-null     object
 4   num_phrases      25 non-null     int64 
 5   skills_text      25 non-null     object
 6   num_words        25 non-null     int64 
 7   unique_words     25 non-null     object
 8   num_uniques      25 non-null     int64 
dtypes: int64(4), object(5)
memory usage: 1.9+ KB


In [4]:
# Flatten unique words lists to single list
all_words = df["unique_words"].sum()

# Get all words
words = list(all_words)

# Count all words
num_words = len(words)

# Get the frequency of each word
word_frequencies = {}

for word in words:
    if word in word_frequencies:
        word_frequencies[word] += 1
    else:
        word_frequencies[word] = 1

# Count unique words
count_all_unique_words = len(word_frequencies)

# Convert dictionary to list of (word, count) tuples
word_counts = list(word_frequencies.items())

# Sort by count in descending order
sorted_frequencies = sorted(word_counts, key=lambda x: x[1], reverse=True)

print(f"All words with repetitions: {words}")
print(f"Frequency of words: {sorted_frequencies}")

All words with repetitions: ['Crypto', 'Stakeholder', 'Exchange', 'Leadership', 'Collaboration', 'Engineering', 'UX', 'Design', 'Development', 'Team', 'Product', 'Management', 'methodologies', 'management', 'Monitoring', 'making', 'improvement', 'Leadership', 'Stress', 'decision', 'Data-driven', 'Communication', 'development', 'knowledge', 'Process', 'Agile', 'Technical', 'E-commerce', 'Team', 'optimization', 'Experience', 'Analytics', 'Research', 'Strategy', 'takes', 'in', 'Advertising', 'Excellent', 'it', 'Thinker', 'Leadership', 'Customer', 'Customer-Centric', 'Communication', 'Execution', 'Market', 'Doing', 'Focus', 'Cross-Functional', 'Technology', 'Collaboration', 'Strategic', 'what', 'Team', 'Product', 'testing', 'management', 'Live', 'design', 'Problem', 'Project', 'Casino', 'development', 'product', 'Usability', 'Game', 'solving', 'and', 'Product', 'business', 'improvement', 'Translating', 'technical', 'requirements', 'needs', 'conceptualization', 'Sales', 'into', 'analysis', 

In [5]:
# Summarize statistics
print("Skills Keyword Statistics:")
print(f"Total keywords in all skills: {num_words}")
print(f"Unique keywords across all skills: {count_all_unique_words}")

# Summary statistics exclude index
df[df.columns[~df.columns.isin(["index"])]].describe()

Skills Keyword Statistics:
Total keywords in all skills: 356
Unique keywords across all skills: 203


,num_phrases,num_words,num_uniques
count,25.000000,25.000000,25.000000
mean,7.720000,16.440000,14.240000
std,3.062134,6.055851,5.018964
min,3.000000,8.000000,8.000000
25%,6.000000,12.000000,10.000000
50%,7.000000,16.000000,14.000000
75%,9.000000,20.000000,17.000000
max,16.000000,31.000000,25.000000


In [6]:
"""# Generate word cloud
wordcloud = WordCloud(width=900, height=380, colormap="summer").generate(df["skills"])

# Plot word cloud
plt.figure(figsize=(16, 6))
plt.imshow(wordcloud, interpolation="bilinear")
plt.title("\nSkills Word Cloud\n", size=18, color="#222222")
plt.axis("off")

# Set the background color of the figure to white
plt.gcf().patch.set_facecolor("white")

# Adjust layout to center the plot
plt.tight_layout()

plt.show()"""

'# Generate word cloud\nwordcloud = WordCloud(width=900, height=380, colormap="summer").generate(df["skills"])\n\n# Plot word cloud\nplt.figure(figsize=(16, 6))\nplt.imshow(wordcloud, interpolation="bilinear")\nplt.title("\nSkills Word Cloud\n", size=18, color="#222222")\nplt.axis("off")\n\n# Set the background color of the figure to white\nplt.gcf().patch.set_facecolor("white")\n\n# Adjust layout to center the plot\nplt.tight_layout()\n\nplt.show()'

In [7]:
from collections import Counter

# Count the frequency of each skill
skill_counts = Counter(skill["name"] for skill in skills)

# Print the most common skills
most_common_skills = skill_counts.most_common()
print(f"Most common skills for {job_title}: {most_common_skills}")

NameError: name 'skills' is not defined

In [ ]:
import re

data = """
name='Crypto Exchange Product Management' reference_text='Completely owning and building our Crypto Exchange product, increasing the number of tokens tradable in the application and providing the tools retail users expect from exchanges'
name='Team Leadership' reference_text="Setting your team's goals, success metrics, and roadmap to align with Revolut’s mission and drive maximum impact based on data analysis, market research, and company strategy"
name='Crypto Product Development' reference_text='Working with our Core Crypto team to expand and improve our suite of crypto products, including deposits, withdrawals, and staking, making them more accessible for our retail users'
name='UX Design' reference_text='Working closely with Design and UX Research to define the customer journey and create an amazing user experience'
name='Engineering Collaboration' reference_text='Liaising with Engineering to ensure effective delivery of the product'
name='Stakeholder Collaboration' reference_text='Collaborating with other stakeholders, such as Product Marketing, Legal, and Product Strategy'
"""

# Extract skills and reference texts
skill_data = re.findall(r"name='(.*?)' reference_text='(.*?)'", data)

# Print extracted skills and reference texts
for skill in skill_data:
    print(f"Skill: {skill[0]}\nReference Text: {skill[1]}\n")

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud

# Assuming skill_data contains the extracted skills
all_skills = " ".join(skill[0] for skill in skill_data)

# Generate word cloud
wordcloud = WordCloud(width=900, height=380, colormap="summer").generate(all_skills)

# Plot word cloud
plt.figure(figsize=(16, 6))
plt.imshow(wordcloud, interpolation="bilinear")
plt.title("\nSkills Word Cloud\n", size=18, color="#222222")
plt.axis("off")

# Set the background color of the figure to white
plt.gcf().patch.set_facecolor("white")

# Adjust layout to center the plot
plt.tight_layout()

plt.show()

In [ ]:
from collections import Counter
import spacy

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Tokenize the text
doc = nlp(all_skills)

# Perform frequency analysis
word_freq = Counter(token.text for token in doc)

# Display the top N most common words
top_n = 5
print(f"Top {top_n} Most Common Words:")
for word, count in word_freq.most_common(top_n):
    print(f"{word}: {count}")

In [ ]:
# Display the top N most common words
top_n = 10
top_words, top_frequencies = zip(*word_freq.most_common(top_n))

# Normalize frequencies to use as sizes for dots
sizes = [freq * 10 for freq in top_frequencies]

# Create a scatter plot
plt.figure(figsize=(16, 4.5))
ax = plt.gca()

# Set the background color of the entire figure
fig = plt.gcf()
fig.patch.set_facecolor("white")

# Plot scatter points
plt.scatter(top_words, top_frequencies, alpha=0.7, edgecolors="w", marker="o")

# Style the plot
plt.style.use("classic")
ax.grid(True)

from matplotlib.colors import LogNorm

# Create a scatter plot with different colors and sizes
scatter = ax.scatter(
    top_words,
    top_frequencies,
    c=top_frequencies,
    cmap="summer",
    s=sizes,
    alpha=0.7,
    edgecolors="w",
    marker="o",
    norm=LogNorm(),
)

# Set plot title with background color
plt.title("\nTop 10 Word Frequency Scatter Plot\n", size=18, color="#222222")

# Label the axes
plt.xlabel("\nWords\n", size=14, color="#222222")
plt.ylabel("\nFrequency\n", size=14, color="#222222")

# Rotate x-axis labels for better readability
plt.xticks(rotation=35, ha="right", color="#333333")
plt.yticks(color="#333333")

# Remove top and right spines
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)

# Set the left and bottom spines to grey color
ax.spines["left"].set_color("grey")
ax.spines["bottom"].set_color("grey")

# Display the plot
plt.show()

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# EXAPLES
job_skills = {
    "Data Scientist": ["Python", "Machine Learning", "Data Analysis", "Statistics"],
    "Software Engineer": ["Java", "JavaScript", "Software Development", "Databases"],
    "Product Manager": ["Product Strategy", "Market Research", "Project Management"],
    "UX Designer": ["User Experience", "Wireframing", "UI Design"],
    "Business Analyst": [
        "Business Analysis",
        "Data Modeling",
        "Requirements Gathering",
    ],
}

In [ ]:
# Placeholder function to generate visualizations
def generate_visualization(job_type):
    clear_output(wait=True)
    print(f"Visualizations for {job_type}:\n")
    print(f"Skills: {', '.join(job_skills.get(job_type, []))}")

    # Get all skills for the selected job type
    all_skills = " ".join(skill[0] for skill in job_skills)

    # Generate word cloud
    wordcloud = WordCloud(width=900, height=380, colormap="summer").generate(all_skills)

    # Plot word cloud
    plt.figure(figsize=(16, 6))
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.title("\nSkills Word Cloud\n", size=18, color="#222222")
    plt.axis("off")

    # Set the background color of the figure to white
    plt.gcf().patch.set_facecolor("white")

    # Adjust layout to center the plot
    plt.tight_layout()

    plt.show()

In [ ]:
# List of job types
job_types = list(job_skills.keys())

# Dropdown widget for job selection
job_dropdown = widgets.Dropdown(
    options=job_types,
    value=job_types[0],
    description="Select Job Type:",
    disabled=False,
    style={"description_width": "initial", "width": "400px"},
)

selected_job_type = job_dropdown.value
skills_for_selected_job = job_skills.get(selected_job_type, [])
all_skills = " ".join(skills_for_selected_job)


# Function to update visualizations when the dropdown value changes
def on_job_type_change(change):
    generate_visualization(change.new)


# Attach the function to the dropdown's observe event
job_dropdown.observe(on_job_type_change, names="value")

# Display the initial visualization
generate_visualization(job_types[0])

# Display the dropdown widget
display(job_dropdown)

In [ ]:
def generate_visualization(job_type):
    clear_output(wait=True)
    print(f"Visualizations for {job_type}:\n")

    # Get skills for the selected job type
    skills_for_selected_job = job_skills.get(job_type, [])
    all_skills = " ".join(skills_for_selected_job)

    # Generate word cloud
    wordcloud = WordCloud(width=900, height=380, colormap="summer").generate(all_skills)

    # Plot word cloud
    plt.figure(figsize=(16, 6))
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.title("\nSkills Word Cloud\n", size=18, color="#222222")
    plt.axis("off")

    # Set the background color of the figure to white
    plt.gcf().patch.set_facecolor("white")

    # Adjust layout to center the plot
    plt.tight_layout()

    plt.show()

In [ ]:
import ipywidgets as widgets
from IPython.display import display
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# EXAMPLES
job_skills = {
    "Data Scientist": ["Python", "Machine Learning", "Data Analysis", "Statistics"],
    "Software Engineer": ["Java", "JavaScript", "Software Development", "Databases"],
    "Product Manager": ["Product Strategy", "Market Research", "Project Management"],
    "UX Designer": ["User Experience", "Wireframing", "UI Design"],
    "Business Analyst": [
        "Business Analysis",
        "Data Modeling",
        "Requirements Gathering",
    ],
}


# Placeholder function to generate visualizations
def generate_visualization(job_type):
    # Get skills for the selected job type
    skills_for_selected_job = job_skills.get(job_type, [])
    all_skills = " ".join(skills_for_selected_job)

    # Generate word cloud
    wordcloud = WordCloud(width=900, height=380, colormap="summer").generate(all_skills)

    # Plot word cloud
    plt.figure(figsize=(16, 6))
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.title("\nSkills Word Cloud\n", size=18, color="#222222")
    plt.axis("off")

    # Set the background color of the figure to white
    plt.gcf().patch.set_facecolor("white")

    # Adjust layout to center the plot
    plt.tight_layout()

    plt.show()


# List of job types
job_types = list(job_skills.keys())

# Dropdown widget for job selection
job_dropdown = widgets.Dropdown(
    options=job_types,
    value=job_types[0],
    description="Select Job Type:",
    disabled=False,
    style={"description_width": "initial", "width": "400px"},
)


# Function to update visualizations when the dropdown value changes
def on_job_type_change(change):
    generate_visualization(change.new)


# Attach the function to the dropdown's observe event
job_dropdown.observe(on_job_type_change, names="value")

# Display the initial visualization
generate_visualization(job_types[0])

# Display the dropdown widget
display(job_dropdown)